Food Delivery Data Anlaysis Project

In [ ]:
import pandas as pd


In [ ]:
orders = pd.read_csv("/content/orders.csv")
users = pd.read_json("/content/users.json")

In [ ]:
conn = sqlite3.connect("food_delivery.db")
cursor = conn.cursor()



In [ ]:
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()



In [ ]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [ ]:
data1 = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [ ]:
final_data = pd.merge(
    data1,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [ ]:
final_data.to_csv("final_food_delivery_dataset.csv", index=False)


Which city has the highest total revenue (total_amount) from Gold members?  
Hyderabad
Bangalore
Chennai
Pune

In [ ]:
import pandas as pd

df = pd.read_csv("/content/final_food_delivery_dataset.csv")

gold_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_revenue


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


Which cuisine has the highest average order value across all orders?
Indian
Chinese
Italian
Mexican

In [ ]:
cuisine_avg = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

cuisine_avg


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
< 500
500 – 1000
1000 – 2000
> 2000

In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

user_total = (
    df.groupby("user_id")["total_amount"]
    .sum()
)

count_users = user_total[user_total > 1000].count()
count_users


np.int64(2544)

Which restaurant rating range generated the highest total revenue?
3.0 – 3.5
3.6 – 4.0
4.1 – 4.5
4.6 – 5.0

In [ ]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

rating_revenue = (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


/tmp/ipython-input-1177544458.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


Among Gold members, which city has the highest average order value?
Hyderabad
Bangalore
Chennai
Pune

In [ ]:
gold_city_aov = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_aov


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
Indian
Chinese
Italian
Mexican

In [ ]:
cuisine_summary = df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values("restaurant_count")

cuisine_summary


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
40%
45%
50%
55%

In [ ]:
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])

percentage = round((gold_orders / total_orders) * 100)
percentage


50

Which restaurant has the highest average order value but less than 20 total orders?
Grand Cafe Punjabi
Grand Restaurant South Indian
Ruchi Mess Multicuisine
Ruchi Foods Chinese

In [ ]:
restaurant_stats = df.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]
filtered.sort_values("avg_order_value", ascending=False)



,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19
...,...,...
Annapurna Tiffins Punjabi,621.828947,19
Darbar Tiffins Non-Veg,596.815556,18
Darbar Restaurant Punjabi,589.972857,14


In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

restaurant_stats = (
    df.groupby("restaurant_name_x")
      .agg(
          avg_order_value=("total_amount", "mean"),
          total_orders=("order_id", "count")
      )
)

result = (
    restaurant_stats[restaurant_stats["total_orders"] < 20]
    .sort_values("avg_order_value", ascending=False)
    .head(1)
)

result



,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13


In [ ]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered_df = df[df["restaurant_name_x"].isin(options)]

restaurant_stats = (
    filtered_df.groupby("restaurant_name_x")
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "count")
    )
)

result = (
    restaurant_stats[restaurant_stats["total_orders"] < 20]
    .sort_values("avg_order_value", ascending=False)
    .head(1)
)

result


,avg_order_value,total_orders
restaurant_name_x,,
Ruchi Foods Chinese,686.603158,19


Which combination contributes the highest revenue?
Gold + Indian cuisine
Gold + Italian cuisine
Regular + Indian cuisine
Regular + Chinese cuisine

In [ ]:
combo_revenue = (
    df.groupby(["membership", "cuisine"])["total_amount"]
      .sum()
      .sort_values(ascending=False)
)

combo_revenue



membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

During which quarter of the year is the total revenue highest?
Q1 (Jan–Mar)
Q2 (Apr–Jun)
Q3 (Jul–Sep)
Q4 (Oct–Dec)

In [ ]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

df["quarter"] = df["order_date"].dt.to_period("Q")

quarter_revenue = (
    df.groupby("quarter")["total_amount"]
      .sum()
      .sort_values(ascending=False)
)

quarter_revenue


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

# 1. Orders with rating >= 4.5
orders_rating_45 = df[df["rating"] >= 4.5]["order_id"].count()

# 2. Orders in top revenue city among Gold members
gold_df = df[df["membership"] == "Gold"]
top_city = gold_df.groupby("city")["total_amount"].sum().idxmax()
orders_top_city_gold = gold_df[gold_df["city"] == top_city]["order_id"].count()

# 3. Total orders by Gold members
total_gold_orders = gold_df["order_id"].count()

# 4. Total revenue from Hyderabad (rounded)
hyd_revenue = round(df[df["city"] == "Hyderabad"]["total_amount"].sum())

# 5. Distinct users with at least one order
distinct_users = df["user_id"].nunique()

# 6. Average order value for Gold members
gold_aov = round(gold_df["total_amount"].mean(), 2)

orders_rating_45, orders_top_city_gold, total_gold_orders, hyd_revenue, distinct_users, gold_aov


(np.int64(3374),
 np.int64(1337),
 np.int64(4987),
 1889367,
 2883,
 np.float64(797.15))